# Introduction

Early morning and having ideas 🙄  This is based on my conversation with Elizabeth Newman and my recent discussion with Mia. 

Can one put convex glasses on a neural network? 😎

# Idea

Support I have a non-linear function $f(x)$, that appoximates some output $y$ given some input $x$.  Then the error function is given by

$$
E = \|f(X) - Y\|^2_F
$$

where $\|\cdot\|_F$ is the Frobenius norm.  We have also abused notation, but in a standard way, by letting $X$ be the matrix of inputs and $Y$ be the matrix of outputs.  Each column $x_i$ of $X$ and column $y_i$ of $Y$ is a training example and $f$ is applied column-wise.

The idea is to add a linear function $G$ that post-processes the output of $f$, such that the error function becomes

$$
E = \sum_{i=1}^k \|G \cdot f(x_i) - y_i\|^2_F. 
$$

where $k$ is the number of training examples.

Note, if $f$ is a *fixed* function then solving for $G$ is a convex problem.  I.e., for each training example $(x_i, y_i)$, we can replace $f(x_i)$ with $\hat{y}_i$
to get 

$$
E_G = \sum_{i=1}^k \|G \cdot \hat{y}_i - y_i\|^2_F. 
$$

In fact, $G$ can be solved for in closed form by using the normal equations!

Solving for $G$ is "free" (since it arises from a convex problem that can be solved in closed form) and we can use it to improve the performance of $f$, at least on the training data.  I.e., we know that

$$
E_G \leq E
$$

So, why don't we always do this?  I mean, given any method on any leaderboard that does not use this trick, we can always improve it by adding this trick.  

Well, one reason might be that solving for $G$ leads to overfitting.  But this seems a little surprising, since $G$ is a linear function.  So, it seems like it would be hard to overfit with a linear function, especially when the dimension of $G$ us the same as the *output* dimension of $f$.  In particular, it is **not a function of the number of unknowns** in $f$.

In fact, this might be why this is not commonly done. For example, when $y_i$ is a scalar, then $G$ is a scalar.  So, we are just adding a constant to the output of $f$.  This is not very interesting.  But, when $y_i$ is a vector, then $G$ is a matrix.  So, we are adding a linear transformation to the output of $f$.  This is more interesting, but there is a bigger idea here I think...🤔

# Bigger Idea

Suppose we have an $f$ which is of a special form. For example, suppose $f$ is a neural network.  Then, we can think of $f$ as a composition of functions

$$
f = f_n \circ f_{n-1} \circ \cdots \circ f_1
$$

where each $f_i$ is a layer of the neural network, or really any set of non-linear functions.

Rewriting the equation for $E_G$ in terms of the composition of functions, we get

$$
E_{G_n} = \sum_{i=1}^k \|G_n \cdot f(x_i) - y_i\|^2_F = \sum_{i=1}^k \|G_n \cdot f_n \circ f_{n-1} \circ \cdots \circ f_1(x_i) - y_i\|^2_F.
$$



Ok, now for a little bit of magic 🪄.  Just for the sake of arguement, consider the following

$$
E_{G_{n-1},G_{n}} = \sum_{i=1}^k \|G_{n-1} \cdot f_{n-1} \circ \cdots \circ f_1(x_i) + G_n \cdot f_n \circ f_{n-1} \circ \cdots \circ f_1(x_i) - y_i\|^2_F.
$$

In the neural network parlance this is like adding a skip connection between the output of $f_{n-1}$ and the output of $f_n$.  But, we are not adding a standard skip connection, we are instead adding a linear transformation to the output of $f_{n-1}$.  So, it is still a convex problem that can be solved in closed form!  

We begin be writing the equation for $E_{G_{n-1},G_{n}}$ in as a matrix equation

$$
E_{G_{n-1},G_{n}} = \sum_{i=1}^k \left| \begin{bmatrix} G_{n-1} & G_n \\ \end{bmatrix} \cdot \begin{bmatrix} f_{n-1} \circ \cdots \circ f_1(x_i) \\ f_n \circ f_{n-1} \circ \cdots \circ f_1(x_i) \\ \end{bmatrix} - y_i \right|^2_F.
$$


And observe that, as before, $f_{n-1} \circ \cdots \circ f_1(x_i)$ and $f_n \circ f_{n-1} \circ \cdots \circ f_1(x_i)$ are fixed functions of $x_i$.  So, we can replace them with 

$$h^{n-1}_i = f_{n-1} \circ \cdots \circ f_1(x_i)$$ 

and 

$$\hat{y}_i = f_n \circ f_{n-1} \circ \cdots \circ f_1(x_i)$$

 to get

$$
E_{G_{n-1},G_{n}} = \sum_{i=1}^k \left| \begin{bmatrix} G_{n-1} & G_n \\ \end{bmatrix} \cdot \begin{bmatrix} h^{n-1}_i \\ \hat{y}_i \\ \end{bmatrix} - y_i \right|^2_F,
$$

or doing the same notation abuse as before

$$
E_{G_{n-1},G_{n}} = \left| \begin{bmatrix} G_{n-1} & G_n \\ \end{bmatrix} \cdot \begin{bmatrix} H^{n-1} \\ \hat{Y}_i \\ \end{bmatrix} - Y \right|^2_F.
$$


So, what have we gained?  Suppose that the $h^{n-1}_i$ are linearly independent of $y_i$, then looking at the normal equations we see that

$$
\begin{bmatrix} G_{n-1} & G_n \\ \end{bmatrix} = \left( \begin{bmatrix} H^{n-1} \\ \hat{Y}_i \\ \end{bmatrix} \begin{bmatrix} H^{n-1} \\ \hat{Y}_i \\ \end{bmatrix}^T \right)^{-1} \begin{bmatrix} H^{n-1} \\ \hat{Y}_i \\ \end{bmatrix} \cdot Y^T,
$$ 

which we can simply slightly to get

$$
\begin{bmatrix} G_{n-1} & G_n \\ \end{bmatrix} = \left( \begin{bmatrix} H^{n-1} \\ \hat{Y}_i \\ \end{bmatrix} \begin{bmatrix} H^{n-1} \\ \hat{Y}_i \\ \end{bmatrix}^T \right)^{-1} \begin{bmatrix} H^{n-1} \cdot Y^T \\ \hat{Y}_i \cdot Y^T \\ \end{bmatrix},
$$ 

and note that, in expectation, we have $\mathbb{E}[ H^{n-1} \cdot Y^T] = \mathbb{0}$ and therefore $\mathbb{E}[ G_{n-1}] = \mathbb{0}$.

Perhaps more importantly, when the $h^{n-1}_i$ are *not* linearly independent of $y_i$ we can use the same trick to computer $G_{n-1}$ and reduce the error.  Remember, computing *both* $G_{n-1}$ and $G_n$ is free, since they are both convex problems that can be solved in closed form.  Perhaps more importantly, we get the *optimal* $G_{n-1}$ and $G_n$ for *any* fixed $f_{n-1}$ and $f_n$.

# Optimal skip connections!

So, now we can go for it!   Consider

$$
E_{G_{n-1},G_{n}} = \left| \begin{bmatrix} G_0 & G_1 & \cdots & G_{n-1} & G_n \\ \end{bmatrix} \cdot \begin{bmatrix} X \\ H^1 \\ \vdots \\ H^{n-1} \\ \hat{Y}_i \\ \end{bmatrix} - Y \right|^2_F.
$$

where $H^i = f_i \circ \cdots \circ f_1(X)$ (e.g., $H^i$ is the output of the $i$-th layer of a neural network).

Then, we can solve for $G_0, G_1, \cdots, G_{n-1}, G_n$ in closed form by using the normal equations.  And, we can do this for *any* fixed $f_0, f_1, \cdots, f_{n-1}, f_n$.

# A bit of philosophy


In some ways, this all makes perfect sense.   You can consider the $G_i$ as a way of "correcting" the output of $f_i$ so that is can be used as part of the prediction of $Y$.  In this sense, the $G_i$ are like "glasses" that correct the output of $f_i$ so that it can be used as part of the prediction of $Y$.  And, the $G_i$ are "convex" glasses, since they are linear transformations.  So, we are putting convex glasses on a neural network.  😎  

Ok, this is cool!   I wrote the following part of the above

```
In some ways, this all makes perfect sense.   You can consider the $G_i$ as a way of "correcting" the output of $f_i$ so that is can be used as part of the prediction of $Y$.
```

and copilot wrote the following

```
In this sense, the $G_i$ are like "glasses" that correct the output of $f_i$ so that it can be used as part of the prediction of $Y$.  And, the $G_i$ are "convex" glasses, since they are linear transformations.  So, we are putting convex glasses on a neural network.  😎  
```

Pretty cool :-)

But, back to the philosophy.  I wonder why this is not done everywhere!?  I mean, this seems to be "free" in the context of neural network training.  Why not always, as your last step, do something like this? 

Also, this seems to have dynamical systems implications.  I mean, everything above holds if each $f_i$ is the same function.  Of course, we are not assuming that $f(f(x)) = f(x)$, but this says that earlier steps of the dynamical system, even though they don't hit the target, can be used to improve the final prediction.  This seems a little surprising to me.  There must be something is the ODE literature about this. 